# Introduction to Calling LLM APIs with Gemini and LangChain 🦜🔗

In this notebook you'll learn how to use LLM APIs through LangChain. We'll use Google's Gemini API as an example. By the end of this notebook, you will know how to make API calls using LangChain, and why we would do that.

## ⚙️ Setup

👉 Run the cell below to load the environment variables from the `.env` we created in the setup challenge:

In [1]:
from dotenv import load_dotenv

load_dotenv() # Load environment variables from .env file

True

👉 Is the ouput of the cell "`True`"? Great! We have now set up a `GOOGLE_API_KEY` environment variable that will be used to authenticate with the Gemini API whenever we need it.

If not, ask a TA for help.

## Making a Simple API Call

In this notebook, we will demonstrate how to:
1. Make an API call using Google's own library.
1. Do the same using LangChain.

## Using Google Generative AI library

In [2]:
from google import genai

In [3]:
client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents="What is the capital of France?",
)

Have a look at the `response`.

In [10]:
# response.candidates[0]

In [11]:
response.candidates[0].content.parts[0].text

'The capital of France is **Paris**.\n'

Do you see how you can get the actual answer out of it?

Fortunately, we can also just use the `.text` attribute to immediately get the answer. Try it out.

In [12]:
response.text

'The capital of France is **Paris**.\n'

Gemini returns its answers in Markdown format. Let's use that!

In [13]:
from IPython.display import Markdown
Markdown(response.text)

The capital of France is **Paris**.


You can also change the generation parameters. This is how you would do it using `google.genai`:

In [24]:
from google import genai
from google.genai import types # We need to import types for the config

client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents="Write a social media post about how much you're learning about transformers.",
    config=types.GenerateContentConfig(
        max_output_tokens=300,
        temperature=1
    )
)

In [25]:
Markdown(response.text)

Okay, here's a social media post about learning about transformers, with a few variations to suit different platforms and tones:

**Option 1: (General - Twitter/X)**

> Diving deep into the world of Transformers! 🤯 Honestly, my brain feels like it's learning a new language. Attention mechanisms, positional encoding... it's a lot, but SO fascinating. #Transformers #MachineLearning #NLP #AI #DeepLearning Learning something new every day! 🤓

**Option 2: (Slightly more technical - LinkedIn)**

> Spending my time lately exploring the intricacies of Transformer networks.  The architecture is truly elegant, and the impact they've had on NLP and other fields is undeniable. I'm particularly interested in [mention a specific aspect you're studying, e.g., "the application of transformers to time series forecasting"].  Always striving to expand my knowledge in the AI/ML space! #AI #MachineLearning #DeepLearning #Transformers #NLP #ArtificialIntelligence #NeuralNetworks

**Option 3: (More casual - Instagram/Facebook - Include a relevant image like a diagram of a transformer or you studying)**

> Just me and my new best friend: the Transformer model! 😂 Seriously, I'm trying to wrap my head around these things, and it's both challenging and incredibly rewarding. Anyone else find the self-attention mechanism mind-blowing?  Send help (and maybe some good resources

Cool. But imagine you want to try out another API, for example OpenAI's, or Anthropic's?

You'd have to dive into their documentation, and rewrite all your code to use their API. Of course it will be similar, but not the same.

Fortunately we have LangChain!

## Using LangChain 🦜🔗

Why would you use LangChain?

1. **Model-Agnostic Code**

   LangChain provides abstractions that let you swap between different LLM providers (Google, OpenAI, Anthropic, etc.) with minimal code changes. If you write code directly against the Google API, switching providers would require significant refactoring.

2. **Unified Interface**

   LangChain standardizes interactions across different LLM providers, offering consistent methods and response formats regardless of the underlying API.

3. **Composability**

   LangChain's chain and pipeline architecture makes it easier to build complex workflows combining prompts, memory, and retrieval without handling all the plumbing yourself.

4. **Built-in Tools**

   LangChain includes tools for output parsing, prompt templates, and other utilities that you'd otherwise need to implement yourself.

Head to [LangChain's Python API Reference](https://python.langchain.com/api_reference/) and look at the list of integrations in the table of contents on the left. Can you find your favourite LLM provider?

In our code we don't want to use `chat_models.ChatGoogleGenerativeAI` because that is specifically made for Gemini. If we'd ever want to change the LLM, we'd have to change the way we instantiate the model. Fortunately LangChain offers a more generic way to instantiate a model.

Let's use Gemini again, but now using LangChain's generic Chat Models. 

👉 Head over to this page in the [LangChain's documentation](https://python.langchain.com/docs/tutorials/llm_chain/) and find how to instantiate a chat model using Gemini. 

Hints: 
1. Head immediately to the section "Using Language Models".
1. You can select the model you want to use to immediately see the correct documentation.

In [29]:
# YOUR CODE HERE
! pip install -qU "langchain[google-genai]"

In [30]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.5-flash", model_provider="google_genai")


The most basic utilisation of the model is to just use `.invoke()` method:

In [43]:
# ?init_chat_model

In [36]:
# YOUR CODE HERE
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("Translate the following from English into Italian"),
    HumanMessage("hi!"),
]

response = model.invoke(messages)

Let's have a look at the response. We use `pprint()` to pretty print the object's `__dict__` which contains all its attributes and methods.

In [37]:
from pprint import pprint
pprint(response.__dict__)

{'additional_kwargs': {},
 'content': '**Ciao!**',
 'example': False,
 'id': 'run--a3dc1da8-6805-4318-958c-0909be1a21f6-0',
 'invalid_tool_calls': [],
 'name': None,
 'response_metadata': {'finish_reason': 'STOP',
                       'model_name': 'gemini-2.5-flash',
                       'prompt_feedback': {'block_reason': 0,
                                           'safety_ratings': []},
                       'safety_ratings': []},
 'tool_calls': [],
 'type': 'ai',
 'usage_metadata': {'input_token_details': {'cache_read': 0},
                    'input_tokens': 10,
                    'output_token_details': {'reasoning': 34},
                    'output_tokens': 4,
                    'total_tokens': 48}}


In [39]:
# ?pprint

Extract the answer, and display it. Remember it's Markdown, so you can make it appear pretty.

In [40]:
# YOUR CODE HERE
Markdown(response.content)

**Ciao!**

You can check the model's temperature accessing the `.temperature` attribute. Try it out:

In [41]:
# YOUR CODE HERE
model.temperature

0.7

We can also set the model's generation parameters before using it, by simply assigning a new value to the attributes.

Try to code the equivalent of what we did using Google's library before to write a social media post:

In [44]:
?model.invoke

Signature:
model.invoke(
    input: 'LanguageModelInput',
    config: 'Optional[RunnableConfig]' = None,
    *,
    code_execution: 'Optional[bool]' = None,
    stop: 'Optional[list[str]]' = None,
    **kwargs: 'Any',
) -> 'BaseMessage'
Docstring:
Enable code execution. Supported on: gemini-1.5-pro, gemini-1.5-flash,
gemini-2.0-flash, and gemini-2.0-pro. When enabled, the model can execute
code to solve problems.
File:      ~/.pyenv/versions/3.12.9/envs/langchain-env/lib/python3.12/site-packages/langchain_google_genai/chat_models.py
Type:      method

In [52]:


config_0 = types.GenerateContentConfig(
        max_output_tokens=200,
        temperature=1)

model_0 = init_chat_model("gemini-2.0-flash", model_provider="google_genai", configurable_fields=config_0)

messages = [HumanMessage("Write a social media post about how much you're learning about transformers.")]

result_0 = model_0.invoke(messages)

In [54]:
Markdown(result_0.content)

Okay, here's a social media post about learning about transformers:

**Option 1 (Enthusiastic & General):**

Just dove headfirst into the world of Transformers in NLP, and WOW! 🤯 It's like unlocking a whole new level of understanding how machines process language. So much to learn about attention mechanisms, embeddings, and the magic behind models like BERT and GPT. Feeling a little overwhelmed but incredibly excited! #NLP #Transformers #MachineLearning #DeepLearning #AI #LearningJourney #AttentionIsAllYouNeed

**Option 2 (A bit more specific & technical):**

Spending my weekend deep-diving into the architecture of Transformers. Really fascinating how the self-attention mechanism allows the model to weigh the importance of different words in a sequence. Still grappling with the positional encoding, but I'm getting there! Anyone have any resources they found particularly helpful? #Transformers #NLP #SelfAttention #PositionalEncoding #DeepLearning #AI #MachineLearning #Study

**Option 3 (Humorous & relatable):**

My brain currently feels like it's trying to transform itself into a Transformer model. 🤖 So much to learn! Just when I think I understand multi-head attention, I get hit with another layer of complexity. Wish me luck as I continue this journey! 😅 #Transformers #NLP #MachineLearning #DeepLearning #AI #LearningIsHard #SendHelp #BrainsAreHard

**Option 4 (Focus on a specific application):**

Really excited about the potential of Transformers for [mention a specific application like text summarization, translation, or question answering]. Seeing how these models can generate coherent and contextually relevant text is mind-blowing. I'm experimenting with [mention a specific library or tool] and already seeing promising results! #Transformers #NLP #TextSummarization #MachineTranslation #QuestionAnswering #AI #DeepLearning #MachineLearning

**Key things to consider when posting:**

*   **Your audience:** Who are you trying to reach with your post? Adjust the technical level accordingly.
*   **Your personality:** Make sure the post reflects your own voice and style.
*   **Hashtags:** Use relevant hashtags to increase visibility.
*   **Engagement:** Ask a question to encourage interaction.
*   **Visuals:** Consider adding an image or GIF to make your post more engaging.  A diagram of a transformer, a funny AI meme, or even just a picture of you studying can help!

The advantage of this? This LangChain Chat Model can support many other APIs.

The only things you'd need to change to switch to another model:
1. Get an API key for the other model and define it in your code.
1. When you instantiate the model switch out the model and provider.

### Multiple messages

Using the `.invoke()` function with just one message is a bit limiting.

You can provide multiple messages, such as:
- `SystemMessage`, or system messages: to tell the model how to behave
- `HumanMessage`, or User messages: the input from the user
- `AIMessage` or Assistant messages: a response from the model

Let's make a social media writer.

We'll pass the model a system message to explain how to behave. Then in the user message, we can limit ourselves to just give it the subject to write about.

Check the [LangChain documentation](https://python.langchain.com/docs/tutorials/llm_chain/) again to find out how to do this. Look for the section "Using Language Models" again.

Need inspiration for the system message? Here's a basic instruction to get you started:

```python
"""You are a creative social media writer writing posts for a Gen AI student. 
Your posts always include a pun, and a call to action.
Your posts are maximum 200 characters long.
You always use emojis.
"""
```

In [59]:
messages_1 = [SystemMessage("""You are a creative social media writer writing posts for a Gen AI student. 
Your posts always include a pun, and a call to action.
Your posts are minimum 200 characters long.
You always use emojis.
"""), HumanMessage("Write a post about my experinces as an Gen AI student.")]


response_2 = model_0.invoke(messages_1)
# Import the necessary classes

# YOUR CODE HERE

# Create a list of messages

# YOUR CODE HERE

# Generate a response using the list of messages

# YOUR CODE HERE

# Display the response

# YOUR CODE HERE


In [60]:
Markdown(response_2.content)

Okay, here are a few options for your social media posts about your experiences as a Gen AI student, each with a pun, call to action, and emojis:

**Option 1 (Focus on Learning):**

> Diving deep into the world of Gen AI! 🤖 It's a neural-ly exciting journey, packed with learning new things every day. From understanding algorithms to building creative applications, it's definitely not artificial! 😉 What are you waiting for? Learning AI is the future, and the future is now! 🚀 What are your favorite AI resources? Share them in the comments! 👇 #GenAI #AIStudent #MachineLearning #ArtificialIntelligence #FutureTech #StudyAI

**Option 2 (Focus on Challenges):**

> Being a Gen AI student isn't always a smooth algorithm... sometimes you face tough bugs! 🐛 But debugging and problem-solving is part of the fun! It's all about learning and growing! 🌱 Ready to take on the challenge? 💪 Let me know what challenges you face in the field! #GenAIStudent #AIChallenges #Debugging #MachineLearning #KeepLearning #TechLife #ArtificialIntelligence

**Option 3 (Focus on Excitement):**

> My brain is processing at lightning speed as a Gen AI student! ⚡ The possibilities with AI are absolutely infinite, and I'm so excited to be a part of this revolution! 🤩 It's an exponential adventure! 🚀 Ready to join the AI party? 🎉 What AI projects are you working on? Tell me all about them in the comments! 👇 #GenAI #AIRevolution #Innovation #FutureIsNow #MachineLearning #ArtificialIntelligence #Excited #TechStudent

🏁 Congratulations! You now master basic prompting using LangChain, using multiple messages.